In [24]:

import os
import json
import numpy as np
import signal
import time
from pathlib import Path
from pyleecan.Functions.load import load
from pyleecan.Functions.save import save
from pyleecan.definitions import DATA_DIR
from pyleecan.Classes.MachineIPMSM import MachineIPMSM
from pyleecan.Classes.Simu1 import Simu1
from pyleecan.Classes.OPdq import OPdq
from pyleecan.Classes.MagFEMM import MagFEMM
from pyleecan.Classes.InputCurrent import InputCurrent
import matplotlib.pyplot as plt

In [25]:
"""
 analyse et amélioration des machines non simulées
Ce script analyse les 84 machines qui n'ont pas pu être simulées,
compare leurs paramètres avec la machine de référence Toyota Prius,
et les améliore pour qu'elles soient dans des plages acceptables.
"""
class MachineAnalyzer:
    """Classe pour analyser et améliorer les machines non simulées"""
    
    def __init__(self):
        self.reference_machine = None
        self.non_simulated_machines = []
        self.improved_machines = []
        self.analysis_results = {}
        
    def load_reference_machine(self):
        """Charge la machine de référence Toyota Prius"""
        try:
            self.reference_machine = load(os.path.join(DATA_DIR, "Machine", "Toyota_Prius.json"))
            print(" Machine de référence Toyota Prius chargée avec succès")
            return True
        except Exception as e:
            print(f" Erreur lors du chargement de la machine de référence: {e}")
            return False
            
    def get_project_root(self):
        """Retourne le chemin racine du projet"""
        current_file = Path(__file__) if '__file__' in globals() else Path.cwd()
        # dossier contenant 84_machine_non_simuler
        for parent in [current_file] + list(current_file.parents):
            if (parent / "84_machine_non_simuler").exists():
                return parent
        return current_file

    def load_non_simulated_machines(self, directory=None):
        """Charge toutes les machines non simulées"""
        try:
            if directory is None:
                project_root = self.get_project_root()
                non_simulated_dir = project_root / "84_machine_non_simuler"
            else:
                non_simulated_dir = Path(directory)
            
            if not non_simulated_dir.exists():
                print(f" Le répertoire {non_simulated_dir} n'existe pas")
                print(f"   Chemin recherché: {non_simulated_dir.absolute()}")
                return False
                
            json_files = list(non_simulated_dir.glob("*.json"))
            print(f" {len(json_files)} fichiers JSON trouvés dans {non_simulated_dir}")
            
            for json_file in json_files:
                try:
                    machine = load(str(json_file))
                    self.non_simulated_machines.append({
                        'file': json_file,
                        'machine': machine,
                        'name': machine.name
                    })
                except Exception as e:
                    print(f" Erreur lors du chargement de {json_file}: {e}")
                    
            print(f" {len(self.non_simulated_machines)} machines non simulées chargées")
            return True
        except Exception as e:
            print(f" Erreur lors du chargement des machines non simulées: {e}")
            return False
    
    def analyze_machine_parameters(self, machine):
        """Analyse les paramètres d'une machine par rapport à la référence"""
        try:
            comparison = {
                'stator': {},
                'rotor': {},
                'ntcoil': {}
            }
            
            # Paramètres stator
            stator_params = ['H0', 'W0']
            for param in stator_params:
                original_value = getattr(machine.stator.slot, param)
                reference_value = getattr(self.reference_machine.stator.slot, param)
                
                # Tolérance de ±50% par rapport à la référence
                tolerance = 0.5
                min_val = reference_value * (1 - tolerance)
                max_val = reference_value * (1 + tolerance)
                
                is_critical = not (min_val <= original_value <= max_val)
                
                comparison['stator'][param] = {
                    'original_value': original_value,
                    'reference_value': reference_value,
                    'is_critical': is_critical,
                    'min_val': min_val,
                    'max_val': max_val
                }
            
            # Paramètres rotor 
            rotor_params = ['H0', 'W0']
            for param in rotor_params:
                # Vérification que rotor.hole existe et n'est pas vide
                if hasattr(machine.rotor, 'hole') and len(machine.rotor.hole) > 0:
                    original_value = getattr(machine.rotor.hole[0], param)
                else:
                    print(f" Aucun trou trouvé dans le rotor pour {machine.name}")
                    continue
                    
                if hasattr(self.reference_machine.rotor, 'hole') and len(self.reference_machine.rotor.hole) > 0:
                    reference_value = getattr(self.reference_machine.rotor.hole[0], param)
                else:
                    print(f" Aucun trou trouvé dans le rotor de référence")
                    continue
                
                tolerance = 0.5
                min_val = reference_value * (1 - tolerance)
                max_val = reference_value * (1 + tolerance)
                
                is_critical = not (min_val <= original_value <= max_val)
                
                comparison['rotor'][param] = {
                    'original_value': original_value,
                    'reference_value': reference_value,
                    'is_critical': is_critical,
                    'min_val': min_val,
                    'max_val': max_val
                }
            
            # Ntcoil
            original_ntcoil = machine.stator.winding.Ntcoil
            reference_ntcoil = self.reference_machine.stator.winding.Ntcoil
            
            # Plage basée sur la référence ±30%
            tolerance = 0.3
            min_ntcoil = max(6, int(reference_ntcoil * (1 - tolerance)))
            max_ntcoil = int(reference_ntcoil * (1 + tolerance))
            
            is_critical = not (min_ntcoil <= original_ntcoil <= max_ntcoil)
            
            comparison['ntcoil'] = {
                'original_value': original_ntcoil,
                'reference_value': reference_ntcoil,
                'is_critical': is_critical,
                'min_val': min_ntcoil,
                'max_val': max_ntcoil
            }
            
            return comparison
        except Exception as e:
            print(f" Erreur lors de l'analyse des paramètres: {e}")
            import traceback
            traceback.print_exc()
            return None
    
    def compare_with_reference(self, machine_params, ref_params):
        """Compare les paramètres d'une machine avec la machine de référence"""
        comparison = {}
        
        # Comparaison stator
        stator_comparison = {}
        for param, value in machine_params['stator'].items():
            ref_value = ref_params['stator'][param]
            if ref_value != 0:
                deviation = ((value - ref_value) / ref_value) * 100
                stator_comparison[param] = {
                    'machine_value': value,
                    'reference_value': ref_value,
                    'deviation_percent': deviation,
                    'is_critical': abs(deviation) > 50  # Seuil critique à 50%
                }
            else:
                stator_comparison[param] = {
                    'machine_value': value,
                    'reference_value': ref_value,
                    'deviation_percent': 0,
                    'is_critical': False
                }
        
        # Comparaison rotor
        rotor_comparison = {}
        for param, value in machine_params['rotor'].items():
            ref_value = ref_params['rotor'][param]
            if ref_value != 0:
                deviation = ((value - ref_value) / ref_value) * 100
                rotor_comparison[param] = {
                    'machine_value': value,
                    'reference_value': ref_value,
                    'deviation_percent': deviation,
                    'is_critical': abs(deviation) > 50
                }
            else:
                rotor_comparison[param] = {
                    'machine_value': value,
                    'reference_value': ref_value,
                    'deviation_percent': 0,
                    'is_critical': False
                }
        
        comparison['stator'] = stator_comparison
        comparison['rotor'] = rotor_comparison
        
        return comparison
    
    def improve_machine_parameters(self, machine, comparison, machine_index):
        """Améliore les paramètres d'une machine basé sur la comparaison"""
        try:
            # Copie de la machine
            improved_machine = machine.copy()
            
            # Paramètres stator à améliorer
            for param, comp in comparison['stator'].items():
                if comp['is_critical']:
                    # Valeur proche de la référence avec une petite variation
                    ref_value = comp['reference_value']
                    if param in ['H0', 'W0']:  # Paramètres critiques
                        # Ajuster vers la référence avec une petite variation aléatoire
                        variation = np.random.uniform(0.95, 1.05)
                        new_value = ref_value * variation
                        setattr(improved_machine.stator.slot, param, new_value)
            
            # Paramètres rotor à améliorer - CORRECTION ICI
            for param, comp in comparison['rotor'].items():
                if comp['is_critical']:
                    ref_value = comp['reference_value']
                    if param in ['H0', 'W0']:  # Paramètres critiques
                        variation = np.random.uniform(0.95, 1.05)
                        new_value = ref_value * variation
                        # Vérifier que rotor.hole existe et n'est pas vide
                        if hasattr(improved_machine.rotor, 'hole') and len(improved_machine.rotor.hole) > 0:
                            setattr(improved_machine.rotor.hole[0], param, new_value)
            
            #  Ntcoil Ajuster
            if comparison['ntcoil']['is_critical']:
                if improved_machine.stator.winding.Ntcoil > 11:
                    improved_machine.stator.winding.Ntcoil = 11
                    print(f"     Ntcoil ajusté: {comparison['ntcoil']['original_value']} → 11 (plage: 6-11)")
            
            #  nom unique avec l'index
            improved_machine.name = self.generate_machine_name(improved_machine, machine.name, machine_index)
            
            return improved_machine
        except Exception as e:
            print(f" Erreur lors de l'amélioration de la machine: {e}")
            import traceback
            traceback.print_exc()
            return None
    
    def generate_machine_name(self, improved_machine, original_name, machine_index):
        """Génère un nom descriptif unique pour la machine améliorée"""
        try:
            # Extraire les paramètres clés
            stator_H0 = int(improved_machine.stator.slot.H0 * 10000)
            stator_W0 = int(improved_machine.stator.slot.W0 * 10000)
            rotor_H0 = int(improved_machine.rotor.hole[0].H0 * 10000)
            rotor_W0 = int(improved_machine.rotor.hole[0].W0 * 10000)
            ntcoil = improved_machine.stator.winding.Ntcoil
            
            # type de machine
            if "defect" in original_name.lower():
                machine_type = "defect"
            else:
                machine_type = "healthy"
            
            # Format avec identifiant uniquessss
            name = f"Toyota_Prius_{machine_type}_{machine_index:02d}_H{stator_H0}W{stator_W0}RH{rotor_H0}RW{rotor_W0}N{ntcoil}"
            
            return name
        except Exception as e:
            print(f" Erreur lors de la génération du nom: {e}")
            return f"{original_name}_improved_{machine_index:02d}"

    def validate_machine_parameters(self, machine, reference_machine):
        """Valide que les paramètres d'une machine sont proches de la référence"""
        try:
            # Tolérance de ±50% par rapport à la référence
            tolerance = 0.5
            
            # Paramètres de référence
            ref_stator_H0 = reference_machine.stator.slot.H0
            ref_stator_W0 = reference_machine.stator.slot.W0
            
            # Vérifier que les trous du rotor existent
            if hasattr(reference_machine.rotor, 'hole') and len(reference_machine.rotor.hole) > 0:
                ref_rotor_H0 = reference_machine.rotor.hole[0].H0
                ref_rotor_W0 = reference_machine.rotor.hole[0].W0
            else:
                print(" Aucun trou trouvé dans le rotor de référence")
                return False
                
            if hasattr(machine.rotor, 'hole') and len(machine.rotor.hole) > 0:
                machine_rotor_H0 = machine.rotor.hole[0].H0
                machine_rotor_W0 = machine.rotor.hole[0].W0
            else:
                print(" Aucun trou trouvé dans le rotor de la machine")
                return False
                
            ref_ntcoil = reference_machine.stator.winding.Ntcoil
            
            # Plages de validation basées sur la référence
            stator_H0_range = (ref_stator_H0 * (1 - tolerance), ref_stator_H0 * (1 + tolerance))
            stator_W0_range = (ref_stator_W0 * (1 - tolerance), ref_stator_W0 * (1 + tolerance))
            rotor_H0_range = (ref_rotor_H0 * (1 - tolerance), ref_rotor_H0 * (1 + tolerance))
            rotor_W0_range = (ref_rotor_W0 * (1 - tolerance), ref_rotor_W0 * (1 + tolerance))
            ntcoil_range = (max(6, int(ref_ntcoil * (1 - 0.3))), int(ref_ntcoil * (1 + 0.3)))
            
            # Vérifier chaque paramètre
            stator_H0_ok = stator_H0_range[0] <= machine.stator.slot.H0 <= stator_H0_range[1]
            stator_W0_ok = stator_W0_range[0] <= machine.stator.slot.W0 <= stator_W0_range[1]
            rotor_H0_ok = rotor_H0_range[0] <= machine_rotor_H0 <= rotor_H0_range[1]
            rotor_W0_ok = rotor_W0_range[0] <= machine_rotor_W0 <= rotor_W0_range[1]
            ntcoil_ok = ntcoil_range[0] <= machine.stator.winding.Ntcoil <= ntcoil_range[1]
            
            # Afficher les détails de validation
            print(f"    Validation basée sur référence:")
            print(f"      Stator H0: {machine.stator.slot.H0:.6f} (plage: {stator_H0_range[0]:.6f} - {stator_H0_range[1]:.6f})")
            print(f"      Stator W0: {machine.stator.slot.W0:.6f} (plage: {stator_W0_range[0]:.6f} - {stator_W0_range[1]:.6f})")
            print(f"      Rotor H0: {machine_rotor_H0:.6f} (plage: {rotor_H0_range[0]:.6f} - {rotor_H0_range[1]:.6f})")
            print(f"      Rotor W0: {machine_rotor_W0:.6f} (plage: {rotor_W0_range[0]:.6f} - {rotor_W0_range[1]:.6f})")
            print(f"      Ntcoil: {machine.stator.winding.Ntcoil} (plage: {ntcoil_range[0]} - {ntcoil_range[1]})")
            
            return all([stator_H0_ok, stator_W0_ok, rotor_H0_ok, rotor_W0_ok, ntcoil_ok])
        except Exception as e:
            print(f" Erreur lors de la validation: {e}")
            import traceback
            traceback.print_exc()
            return False

    def run_analysis(self):
        """Lance l'analyse complète"""
        try:
            print(" Début de l'analyse des machines non simulées")
            print("=" * 60)
            
            # Chargement de la machine de référence
            if not self.load_reference_machine():
                return False
            
            # paramètres de référence
            print(f"\n Paramètres de référence Toyota Prius:")
            print(f"  Stator H0: {self.reference_machine.stator.slot.H0:.6f} m")
            print(f"  Stator W0: {self.reference_machine.stator.slot.W0:.6f} m")
            print(f"  Rotor H0: {self.reference_machine.rotor.hole[0].H0:.6f} m")
            print(f"  Rotor W0: {self.reference_machine.rotor.hole[0].W0:.6f} m")
            print(f"  Ntcoil: {self.reference_machine.stator.winding.Ntcoil}")
            
            # Charger les machines non simulées
            if not self.load_non_simulated_machines():
                return False
            
            print(f"\n Analyse de {len(self.non_simulated_machines)} machines non simulées...")
            
            # Analyse et amélioration de chaque machine
            for i, machine_data in enumerate(self.non_simulated_machines):
                machine = machine_data['machine']
                print(f"\n--- Machine {i+1}/{len(self.non_simulated_machines)}: {machine.name} ---")
                
                # Analyse des paramètres
                comparison = self.analyze_machine_parameters(machine)
                if comparison is None:
                    continue
                
                # paramètres critiques
                critical_count = 0
                critical_params = []
                
                for param_type in ['stator', 'rotor']:
                    for param, comp in comparison[param_type].items():
                        if comp['is_critical']:
                            critical_count += 1
                            critical_params.append(f"{param_type}_{param}")
                
                if comparison['ntcoil']['is_critical']:
                    critical_count += 1
                    critical_params.append("ntcoil")
                
                print(f"  Paramètres critiques: {critical_count}")
                if critical_params:
                    print(f"  {', '.join(critical_params)}")
                
                # Amélioration de la machine
                improved_machine = self.improve_machine_parameters(machine, comparison, i+1)
                
                if improved_machine:
                    # Validation des machines améliorée
                    is_valid = self.validate_machine_parameters(improved_machine, self.reference_machine)
                    
                    if is_valid:
                        print(f" Machine améliorée validée: {improved_machine.name}")
                        self.improved_machines.append({
                            'original': machine,
                            'improved': improved_machine,
                            'comparison': comparison
                        })
                    else:
                        print(f" Machine améliorée non validée: {improved_machine.name}")
                else:
                    print(f" Impossible d'améliorer {machine.name}")
            
            # Résumé
            print(f"\n Résumé de l'analyse:")
            print(f"  Machines analysées: {len(self.non_simulated_machines)}")
            print(f"  Machines améliorées: {len(self.improved_machines)}")
            print(f"  Taux de succès: {(len(self.improved_machines)/len(self.non_simulated_machines)*100):.1f}%")
            
            return True
        except Exception as e:
            print(f" Erreur lors de l'analyse: {e}")
            return False

    
    def save_improved_machines(self, output_dir="84_machine_improved"):
        """Sauvegarde les machines améliorées"""
        try:
            output_path = Path(output_dir)
            output_path.mkdir(exist_ok=True)
            
            print(f"\n Sauvegarde des machines améliorées dans {output_dir}...")
            
            for i, machine_data in enumerate(self.improved_machines):
                improved_machine = machine_data['improved']
                filename = f"{improved_machine.name}.json"
                filepath = output_path / filename
                
                try:
                    save(improved_machine, str(filepath))
                    print(f"    {filename} sauvegardée")
                except Exception as e:
                    print(f"  Erreur lors de la sauvegarde de {filename}: {e}")
            
            print(f" {len(self.improved_machines)} machines améliorées sauvegardées")
            return True
        except Exception as e:
            print(f" Erreur lors de la sauvegarde: {e}")
            return False

    def generate_analysis_report(self, output_file="analysis_report.txt"):
        """Génère un rapport d'analyse détaillé"""
        try:
            with open(output_file, 'w', encoding='utf-8') as f:
                f.write("RAPPORT D'ANALYSE DES MACHINES NON SIMULÉES\n")
                f.write("=" * 50 + "\n\n")
                
                f.write(f"Nombre de machines analysées: {len(self.non_simulated_machines)}\n")
                f.write(f"Nombre de machines améliorées: {len(self.improved_machines)}\n")
                f.write(f"Taux de succès: {len(self.improved_machines)/len(self.non_simulated_machines)*100:.1f}%\n\n")
                
                f.write("DÉTAIL DES AMÉLIORATIONS:\n")
                f.write("-" * 30 + "\n")
                
                for i, machine_data in enumerate(self.improved_machines):
                    f.write(f"\n{i+1}. {machine_data['improved'].name}\n")
                    f.write(f"   Paramètres critiques: {len(machine_data['critical_params'])}\n")
                    if machine_data['critical_params']:
                        f.write(f"   {', '.join(machine_data['critical_params'])}\n")
            
            print(f" Rapport d'analyse généré: {output_file}")
            return True
        except Exception as e:
            print(f" Erreur lors de la génération du rapport: {e}")
            return False

def main():
    """Fonction principale"""
    print(" ANALYSEUR DE MACHINES NON SIMULÉES")
    print("=" * 60)
    print("Ce script analyse et améliore les 84 machines qui n'ont pas pu être simulées")
    print("en les comparant avec la machine de référence Toyota Prius.\n")
    
    # Création de l'analyseur
    analyzer = MachineAnalyzer()
    
    if analyzer.run_analysis():
        # Save machines améliorées
        analyzer.save_improved_machines()
        
        # bref rapport
        analyzer.generate_analysis_report()
        
        print("\n Analyse terminée avec succès!")
        print("Les machines améliorées ont été sauvegardées dans le dossier '84_machine_improved'")
    else:
        print("\n L'analyse a échoué")

if __name__ == "__main__":
    main()

 ANALYSEUR DE MACHINES NON SIMULÉES
Ce script analyse et améliore les 84 machines qui n'ont pas pu être simulées
en les comparant avec la machine de référence Toyota Prius.

 Début de l'analyse des machines non simulées
 Machine de référence Toyota Prius chargée avec succès

 Paramètres de référence Toyota Prius:
  Stator H0: 0.001000 m
  Stator W0: 0.001930 m
  Rotor H0: 0.010960 m
  Rotor W0: 0.042000 m
  Ntcoil: 9
 85 fichiers JSON trouvés dans C:\Users\Dell\Documents\projet_stage\Tutorials\machines_custom_batch_ntcoil\84_machine_non_simuler
 Erreur lors du chargement de C:\Users\Dell\Documents\projet_stage\Tutorials\machines_custom_batch_ntcoil\84_machine_non_simuler\test_results.json: Key "__class__" missing in loaded file
 84 machines non simulées chargées

 Analyse de 84 machines non simulées...

--- Machine 1/84: Toyota_Prius_defect_13_Ntcoil_10 ---
  Paramètres critiques: 1
  stator_H0
    Validation basée sur référence:
      Stator H0: 0.001015 (plage: 0.000500 - 0.001500)
 

In [28]:
class MachineTester:
    """Classe pour tester les machines améliorées"""
    
    def __init__(self):
        self.improved_machines = []
        self.test_results = {}
        self.successful_simulations = []
        self.failed_simulations = []
        self.reference_machine = None
    
    def load_reference_machine(self):
        """Charge la machine de référence Toyota Prius"""
        try:
            # Essayer plusieurs chemins possibles
            possible_paths = [
                "compar_simule_healthy_1_ntcoil_7_machine_ref/Toyota_Prius.json",
                "84_machine_non_simuler/compar_simule_healthy_1_ntcoil_7_machine_ref/Toyota_Prius.json",
                os.path.join(DATA_DIR, "Machine", "Toyota_Prius.json"),
                "Toyota_Prius.json"
            ]
            
            for path in possible_paths:
                if os.path.exists(path):
                    try:
                        self.reference_machine = load(path)
                        print(f"Machine de référence chargée depuis: {path}")
                        return True
                    except Exception as e:
                        print(f"Erreur lors du chargement de {path}: {e}")
                        continue
            
            print("Aucun fichier de référence Toyota_Prius.json trouvé")
            return False
        except Exception as e:
            print(f"Erreur lors du chargement de la machine de référence: {e}")
            return False
    
    def load_improved_machines(self, directory="84_machine_improved"):
        """Charge les machines améliorées"""
        try:
            improved_dir = Path(directory)
            if not improved_dir.exists():
                print(f"Le répertoire {directory} n'existe pas")
                return False
            
            json_files = list(improved_dir.glob("*.json"))
            print(f"{len(json_files)} machines améliorées trouvées")
            
            for json_file in json_files:
                try:
                    machine = load(str(json_file))
                    
                    self.improved_machines.append({
                        'improved': machine,
                        'file': json_file,
                        'name': machine.name
                    })
                except Exception as e:
                    print(f"Erreur lors du chargement de {json_file}: {e}")
            
            print(f"{len(self.improved_machines)} machines améliorées chargées")
            return True
        except Exception as e:
            print(f"Erreur lors du chargement: {e}")
            return False
    
    def create_simulation(self, machine, name="test"):
        """Crée une simulation pour une machine"""
        try:
            # Configuration de la simulation
            simu = Simu1(machine=machine, name=name)
            
            # Point de fonctionnement - Configuration simplifiée
            simu.OP = OPdq(N0=500, Id_ref=0, Iq_ref=0)  # Vitesse réduite
            
            # Module magnétique FEMM - Configuration très simplifiée
            simu.mag = MagFEMM(
                is_periodicity_a=True,
                is_periodicity_t=True,
                is_sliding_band=True,
                Kmesh_fineness=0.3,  # Maillage très grossier
                is_get_mesh=False,
                is_save_FEA=False,
                is_mmfs=False,
                is_mmfr=False,
                is_force=False
            )
            
            # Module courant - Configuration simplifiée
            simu.input = InputCurrent(
                Id_ref=0,
                Iq_ref=0,
                Nt_tot=machine.stator.winding.Ntcoil,
                Nrev=1,
                Nstep=2,  # Très peu d'étapes
                Na_tot=2048
            )
            
            return simu
        except Exception as e:
            print(f"Erreur lors de la création de la simulation: {e}")
            return None
    
    def validate_machine_parameters(self, machine):
        """Valide que les paramètres d'une machine sont dans des plages acceptables"""
        try:
            # Vérifier que les paramètres de base existent
            if not hasattr(machine, 'stator') or not hasattr(machine.stator, 'slot'):
                return False, "Stator ou slot manquant"
            
            if not hasattr(machine, 'rotor') or not hasattr(machine.rotor, 'hole') or len(machine.rotor.hole) == 0:
                return False, "Rotor ou hole manquant"
            
            if not hasattr(machine.stator, 'winding'):
                return False, "Winding manquant"
            
            # Vérifier les valeurs de base
            if machine.stator.slot.H0 <= 0:
                return False, f"Stator H0 invalide: {machine.stator.slot.H0}"
            
            if machine.stator.slot.W0 <= 0:
                return False, f"Stator W0 invalide: {machine.stator.slot.W0}"
            
            if machine.rotor.hole[0].H0 <= 0:
                return False, f"Rotor H0 invalide: {machine.rotor.hole[0].H0}"
            
            if machine.rotor.hole[0].W0 <= 0:
                return False, f"Rotor W0 invalide: {machine.rotor.hole[0].W0}"
            
            if machine.stator.winding.Ntcoil <= 0:
                return False, f"Ntcoil invalide: {machine.stator.winding.Ntcoil}"
            
            return True, "Paramètres valides"
        except Exception as e:
            return False, f"Erreur lors de la validation: {e}"
    
    def test_machine_simulation(self, machine, timeout=60):
        """Teste si une machine peut être simulée avec succès"""
        try:
            # Création de la simulation
            simu = self.create_simulation(machine, f"test_{machine.name}")
            if simu is None:
                return False, "Impossible de créer la simulation"
            
            # Exécution de la simulation avec timeout
            def timeout_handler(signum, frame):
                raise TimeoutError("Simulation timeout")
            
            signal.signal(signal.SIGALRM, timeout_handler)
            signal.alarm(timeout)
            
            try:
                # Exécution de la simulation
                start_time = time.time()
                output = simu.run()
                end_time = time.time()
                signal.alarm(0)  # Annuler le timeout
                
                # Vérification des résultats
                if output is None:
                    return False, "Simulation retourne None"
                
                simulation_time = end_time - start_time
                return True, f"Simulation réussie en {simulation_time:.2f}s"
                
            except TimeoutError:
                signal.alarm(0)
                return False, f"Timeout après {timeout} secondes"
            except Exception as e:
                signal.alarm(0)
                return False, f"Erreur lors de l'exécution: {str(e)}"
                
        except Exception as e:
            return False, f"Erreur lors de la configuration: {str(e)}"
    
    def test_reference_machine_first(self):
        """Teste d'abord la machine de référence pour vérifier la configuration"""
        print("Test de la machine de référence...")
        
        if self.reference_machine is None:
            print("Machine de référence non chargée")
            return False
        
        try:
            # Tester la simulation
            success, message = self.test_machine_simulation(self.reference_machine)
            
            if success:
                print(f"Machine de référence testée avec succès: {message}")
                return True
            else:
                print(f"Erreur avec la machine de référence: {message}")
                return False
                
        except Exception as e:
            print(f"Erreur lors du test de la machine de référence: {e}")
            return False
    
    def test_few_machines_first(self, max_machines=5):
        """Teste quelques machines d'abord"""
        print(f"Test des {max_machines} premières machines...")
        
        success_count = 0
        for i, machine_data in enumerate(self.improved_machines[:max_machines]):
            machine = machine_data['improved']
            print(f"\n--- Test {i+1}/{max_machines}: {machine.name} ---")
            
            try:
                # 1. Validation des paramètres
                is_valid, valid_message = self.validate_machine_parameters(machine)
                if not is_valid:
                    print(f"Paramètres invalides: {valid_message}")
                    continue
                
                # 2. Test de simulation
                success, message = self.test_machine_simulation(machine)
                if success:
                    success_count += 1
                    print(f"Simulation réussie: {message}")
                else:
                    print(f"Simulation échouée: {message}")
                    
            except Exception as e:
                print(f"Erreur lors du test: {e}")
        
        print(f"\nRésultat du test préliminaire:")
        print(f"  Machines testées: {max_machines}")
        print(f"  Simulations réussies: {success_count}")
        print(f"  Taux de succès: {(success_count/max_machines*100):.1f}%")
        
        return success_count > 0
    
    def test_all_machines(self):
        """Teste toutes les machines améliorées"""
        print("Début des tests de simulation")
        print("=" * 60)
        
        for i, machine_data in enumerate(self.improved_machines):
            # Gérer différentes structures de données
            if isinstance(machine_data, dict):
                if 'improved' in machine_data:
                    machine = machine_data['improved']
                elif 'machine' in machine_data:
                    machine = machine_data['machine']
                else:
                    # Si c'est juste la machine directement
                    machine = machine_data
            else:
                # Si c'est directement l'objet machine
                machine = machine_data
                
            print(f"\n--- Test {i+1}/{len(self.improved_machines)}: {machine.name} ---")
            
            # 1. Validation des paramètres
            is_valid, valid_message = self.validate_machine_parameters(machine)
            if not is_valid:
                print(f"   Paramètres invalides: {valid_message}")
                self.failed_simulations.append({
                    'machine': machine,
                    'reason': 'Paramètres invalides',
                    'error': valid_message
                })
                continue
            
            # 2. Test de simulation
            success, message = self.test_machine_simulation(machine)
            
            if success:
                print(f"   Simulation réussie: {message}")
                self.successful_simulations.append({
                    'machine': machine,
                    'message': message
                })
            else:
                print(f"   Simulation échouée: {message}")
                self.failed_simulations.append({
                    'machine': machine,
                    'reason': 'Échec simulation',
                    'error': message
                })
        
        # Résumé des tests
        print(f"\nRésumé des tests:")
        print(f"  Machines testées: {len(self.improved_machines)}")
        print(f"  Simulations réussies: {len(self.successful_simulations)}")
        print(f"  Simulations échouées: {len(self.failed_simulations)}")
        if len(self.improved_machines) > 0:
            success_rate = len(self.successful_simulations)/len(self.improved_machines)*100
            print(f"  Taux de succès: {success_rate:.1f}%")
    
    def save_test_results(self, output_file="test_results.json"):
        """Sauvegarde les résultats des tests"""
        try:
            results = {
                'total_machines': len(self.improved_machines),
                'successful_simulations': len(self.successful_simulations),
                'failed_simulations': len(self.failed_simulations),
                'success_rate': len(self.successful_simulations)/len(self.improved_machines)*100 if len(self.improved_machines) > 0 else 0,
                'successful_machines': [{'name': m['machine'].name, 'message': m['message']} for m in self.successful_simulations],
                'failed_machines': [{'name': m['machine'].name, 'reason': m['reason'], 'error': m['error']} for m in self.failed_simulations]
            }
            
            with open(output_file, 'w', encoding='utf-8') as f:
                json.dump(results, f, indent=2, ensure_ascii=False)
            
            print(f"Résultats sauvegardés dans {output_file}")
            return True
        except Exception as e:
            print(f"Erreur lors de la sauvegarde: {e}")
            return False

def main():
    """Fonction principale"""
    print("TESTEUR DE MACHINES AMÉLIORÉES")
    print("=" * 60)
    print("Ce script teste les machines améliorées pour vérifier qu'elles passent la simulation")
    
    # Création du testeur
    tester = MachineTester()
    
    # Chargement de la machine de référence
    if not tester.load_reference_machine():
        print("Impossible de charger la machine de référence")
        return
    
    # Chargement des machines améliorées
    if not tester.load_improved_machines():
        print("Impossible de charger les machines améliorées")
        return
    
    # Test de la machine de référence d'abord
    if not tester.test_reference_machine_first():
        print("Problème détecté avec la machine de référence - arrêt des tests")
        return
    
    # Test de quelques machines d'abord
    if not tester.test_few_machines_first(5):
        print("Problème détecté avec les machines améliorées - arrêt des tests")
        return
    
    # Test de toutes les machines
    tester.test_all_machines()
    
    # Sauvegarde des résultats
    tester.save_test_results()
    
    print("\nTests terminés!")

if __name__ == "__main__":
    main()

TESTEUR DE MACHINES AMÉLIORÉES
Ce script teste les machines améliorées pour vérifier qu'elles passent la simulation
Machine de référence chargée depuis: C:/Users/Dell/AppData/Local/Programs/Python/Python310/Lib/site-packages/pyleecan/Data\Machine\Toyota_Prius.json
84 machines améliorées trouvées
84 machines améliorées chargées
Test de la machine de référence...
Erreur lors de la création de la simulation: Simu1 class has no "OP" property
Erreur avec la machine de référence: Impossible de créer la simulation
Problème détecté avec la machine de référence - arrêt des tests
